# Laboratorio 2 - Agrupación

## Integrantes - Grupo 19
* Carol Sofía Florido Castro - 202111430
* Juan Martin Vasquez Cristancho - 202113314
* Natalia Villegas Calderon - 202113370


## Objetivos
* Aplicar el proceso de aprendizaje para resolver una tarea de agrupación, desde la preparación de los datos hasta la interpretación de los resultados.
* Aplicar tres algoritmos de clústering (k-means y dos de libre elección) para resolver el objetivo de la organización.
* Determinar los hiperparámetros para la construcción de los modelos dependiendo de los algoritmos utilizados.
* Construir una tabla comparativa que muestre el rendimiento de los tres modelos de agrupación.
* Derivar conclusiones a partir de los mejores grupos identificados, que sean útiles para la organización.
* Comunicar los hallazgos encontrados a la organización, explicando por qué tienen valor para el negocio.

## Caso de Negocio: FinanzasAlpes

La agrupación es una técnica de aprendizaje no supervisado que permite identificar patrones y tendencias en los datos con base en la identificación de grupos o segmentos de objetos con características comunes. Por ejemplo, para el caso una empresa que ofrece servicios de tarjetas de crédito, la segmentación de clientes facilitaría la personalización de productos y servicios de acuerdo con las necesidades y comportamientos específicos de cada grupo identificado. Además, se podrían dirigir campañas de marketing a segmentos particulares, optimizando la comunicación y mejorando la experiencia de los clientes a través de un servicio de atención más ajustado a sus preferencias y hábitos.
En este marco, FinanzasAlpes, una empresa emisora de tarjetas de crédito, busca segmentar a sus clientes según su comportamiento de compra en centros comerciales, con el objetivo de identificar los distintos perfiles de tarjetahabientes y desarrollar estrategias de marketing personalizadas para cada segmento. Para llevar a cabo este estudio, la empresa nos ha contratado como científicos de datos y nos ha proporcionado un conjunto de datos sobre sus clientes, que incluye información detallada sobre saldos, límites de crédito, hábitos de compra y otros aspectos relevantes.



## Datos:
Se trabaja con un conjunto de datos que contiene ### registros de clientes bancarios con los siguientes atributos:

- **ID**: Identificador del cliente.
- **SALDO**: Saldo medio mensual (basado en los saldos medios diarios).
- **F_SALDO**: Relación de los últimos 12 meses con el saldo. (Entre 0 y 1, donde 1: Se actualiza con frecuencia, 0: No se actualiza con frecuencia).
- **COMPRAS**: Importe total de las compras realizadas en los últimos 12 meses.
- **COMPRAS_PUNTUALES**: Importe total de las compras puntuales.
- **COMPRAS_PLAZOS**: Importe total de las compras a plazos.
- **AVANCE_EFECTIVO**: Importe total de avance en efectivo.
- **F_COMPRAS**: Frecuencia de compras (Porcentaje de meses con al menos una compra). (Entre 0 y 1 donde 1: compró con frecuencia, 0: no compró con frecuencia).
- **F_COMPRAS_PUNTUALES**: Frecuencia de las compras puntuales. (Entre 0 y 1, donde 1: compró con frecuencia, 0: no compró con frecuencia).
- **F_COMPRAS_PLAZOS**: Frecuencia de compra a plazos. (Entre 0 y 1, donde 1: compró con frecuencia, 0: no compró con frecuencia).
- **F_AVANCE_EFECTIVO**: Frecuencia de los avances en efectivo. (Entre 0 y 1, donde 1: compró con frecuencia, 0: no compró con frecuencia).
- **P_AVANCE_EFECTIVO**: Importe medio por operación de avance en efectivo.
- **P_COMPRAS**: Importe medio por operación de compra.
- **LÍMITE_CREDITO**: Límite de crédito.
- **PAGOS**: Pagos totales (importe pagado por el cliente para reducir el saldo de su extracto en el periodo).
- **PAGOS_MINIMOS**: Total pagos mínimos en el periodo.
- **F_PAGOS_COMPLETOS**: Porcentaje de meses con pago completo del saldo vencido.
- **MESES_CLIENTE**: Número de meses como cliente.